In [ ]:
# Start your code here!
import os
from openai import OpenAI

# Define the client
client = OpenAI(api_key="<OPENAI_API_TOKEN>")

# Open the customer_call.wav file
audio_file = open("customer_call.wav", "rb")

# Create a transcript from the audio file
response = client.audio.transcriptions.create(model="whisper-1", file=audio_file)
transcript = response.text
print(transcript)

In [ ]:
# Complete a request identify the country code from the transcript
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", 
        "content": f"""Identify the language of the following text and respond only with the country code (e.g. 'en', 'fr', 'uk'): {transcript}"""}]
)

# Extract the country code from the response
country_code = response.choices[0].message.content
print(country_code)

In [ ]:
# Translate the transcript into English
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": f"""Translate this customer transcript from country code {country_code} to English: {transcript}"""}
    ]
)

# Extract the translated transcript text
en_transcript = response.choices[0].message.content
print(en_transcript)

In [ ]:
# Fix the mistake in the transcript
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user",
        "content": f"""You are an AI assistant that corrects transcripts by fixing fixing incorrect technical names and terminology. Please refine the following transcript:\n\n{en_transcript}. Lang Chain should be LangChain"""}
    ]
)

corrected_text = response.choices[0].message.content
print(corrected_text)

In [ ]:
# Send the moderation request
response = client.moderations.create(
    model="text-moderation-latest",
    input=corrected_text
)

# Extract scores and convert to dictionary
scores = response.results[0].category_scores.model_dump()

# Extract harassment score
harassment_score = scores['harassment']

if harassment_score > 0.8:
    print("Content flagged for harassment!")
else:
    print("Content is safe from harassment.")

In [ ]:
# Complete the prompt
instruction_prompt = f"""
#### Role
You are a professional AI support assistant for DataCamp. You help with:
- Sales (pricing, plans, billing)
- Content (courses, recommendations, feedback)
- Marketing (partnerships, collaborations)

#### How to Respond
1. Use the FAQs: {faqs}
2. Use the content overview: {content_overview}
3. Respond clearly and concisely in up to 3 paragraphs.
4. If unsure, direct the user to support@datacamp.com.
"""

# Generate response
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": instruction_prompt},
        {"role": "user", "content": corrected_text}
    ]
)

chatbot_reply = response.choices[0].message.content
print(chatbot_reply)

In [ ]:
# Translate chatbot reply
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": f"""Translate the following text
        from English to language {country_code}. Only return the translated text!"""},
        {"role": "user", "content": chatbot_reply}
    ]
)

# Extract translated text
translated_reply = response.choices[0].message.content
print(translated_reply)

In [ ]:
# Create the text-to-speech request
response = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="onyx",
    input=translated_reply
)

# Stream the response to an MP3 file
response.stream_to_file("output.mp3")